In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# Importing the   data


fake = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Fake.csv')
true = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/True.csv')

fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
# Shape of both dataset


print(fake.shape)
print(true.shape)

(23481, 4)
(21417, 4)


## Preparing the dataset

In [ ]:
# removing unnecessary colums

true = true.drop(['subject', 'date', 'title'], axis=1)
fake = fake.drop(['subject', 'date', 'title'], axis=1)

# labelling the data

true['label'] = 1
fake['label'] = 0


# concatenating both dataframes

df = pd.concat([true, fake])

# shuffling the data

df = df.sample(frac = 1)

# resetting the index values

df.reset_index(inplace=True, drop=True)

In [ ]:
print(df.shape)

df.head()

(44898, 2)


,text,label
0,PRESIDENT TRUMP and FIRST LADY MELANIA TRUMP a...,0
1,We mentioned in an article yesterday that the ...,0
2,BEIRUT (Reuters) - The Syrian government deleg...,1
3,"YANGON/SHAMLAPUR, Bangladesh (Reuters) - Myanm...",1
4,WASHINGTON (Reuters) - Divisions have emerged ...,1


In [ ]:
# checking for null values

df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
## Get the Independent Features

X = df.drop('label',axis=1)

## Get the Dependent features

y = df['label']


print(X.shape)
print(y.shape)

(44898, 1)
(44898,)


In [ ]:
# Importing necessary libraries for embedding and training LSTM model


import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
messages = X.copy()
messages.head()

,text
0,PRESIDENT TRUMP and FIRST LADY MELANIA TRUMP a...
1,We mentioned in an article yesterday that the ...
2,BEIRUT (Reuters) - The Syrian government deleg...
3,"YANGON/SHAMLAPUR, Bangladesh (Reuters) - Myanm..."
4,WASHINGTON (Reuters) - Divisions have emerged ...


In [ ]:
import re
import spacy


# importing stopwords from spacy

nlp = spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words

In [ ]:
# creating a function to preprocess all text

def preprocess(sentence):
    # making all words in lower case
    sentence = sentence.lower()
    # removing URL
    sentence = sentence.replace('https://', ' ')
    # removing all character other than letters
    sentence = re.sub('[^a-zA-Z]',' ',sentence)

    sentence = " ".join(sentence.split())
    
    # remove stop words
    raw = []
    for word in sentence.split():
        if word in stop_words:
            pass
        else:
            raw.append(word)
    sentence = ' '.join(raw)
    # lemmatization
    sentence = ' '.join([i.lemma_ for i in nlp(sentence)])
    
    return sentence

In [ ]:
# creating the corpora

corpus = [text for text in df['text']]

In [ ]:
# preprocessing all the news texts
for i in range(len(corpus)):
    corpus[i] = preprocess(corpus[i])

In [ ]:
type(corpus)

list

In [ ]:
len(corpus)

44898

In [ ]:
# One-hot Encoding and Padding the data

voc_size = 10000

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
corpus[1]

'mention article yesterday real objective recount effort probably find voter fraud strip trump electoral vote traditionally blue win state pa wi mi state process recount effort trump use electoral vote final tally republican majority congress vote allow trump use vote hope likely action throw trump s presidency question voter set illegitimate president mind american citizen good news possiblity recount state pennsylvania electoral vote look good hillary right jill stein s hillary clinton s effort recount presidential vote swing state appear sputter philadelphia stein presidential candidate raise million dollar file recount wisconsin process pennsylvania complicated candidate file recount instead court attempt prove widespread voter fraud s evidence point organize voter initiate recount effort stein s team go ask thousand volunteers pennsylvania mobilize voting division know precinct recount voter division sign file affidavit noon monday philadelphia city board election receive petition

In [ ]:
onehot_repr[1]

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[7950 5262 9152 ... 4270   13 7950]
 [2401 2422 4937 ... 2477 4937 8684]
 [7895 5518  205 ... 1082 8144 3070]
 ...
 [2459 1062 4228 ... 8381 5127 5684]
 [6320 8124 8817 ... 7376 7843 8695]
 [3632 4558 4337 ... 9272 3287 7180]]


# (1) Training model with LSTM

In [ ]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(44898, (44898,))

In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape,y_final.shape

((44898, 20), (44898,))

In [ ]:
# splitting data in to train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
print(X_test.shape)
print(y_test.shape)

(14817, 20)
(14817,)


In [ ]:
# Training the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 [==============================] - 11s 8ms/step - loss: 0.2221 - accuracy: 0.9058 - val_loss: 0.1536 - val_accuracy: 0.9393
Epoch 2/10
471/471 [==============================] - 3s 7ms/step - loss: 0.1004 - accuracy: 0.9627 - val_loss: 0.1393 - val_accuracy: 0.9470
Epoch 3/10
471/471 [==============================] - 3s 7ms/step - loss: 0.0660 - accuracy: 0.9768 - val_loss: 0.1442 - val_accuracy: 0.9484
Epoch 4/10
471/471 [==============================] - 3s 7ms/step - loss: 0.0421 - accuracy: 0.9863 - val_loss: 0.1690 - val_accuracy: 0.9468
Epoch 5/10
471/471 [==============================] - 3s 7ms/step - loss: 0.0258 - accuracy: 0.9918 - val_loss: 0.1907 - val_accuracy: 0.9473
Epoch 6/10
471/471 [==============================] - 3s 7ms/step - loss: 0.0171 - accuracy: 0.9942 - val_loss: 0.2150 - val_accuracy: 0.9476
Epoch 7/10
471/471 [==============================] - 3s 7ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.2478 - val_accuracy: 0.9427
Epoch

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

941/941 [==============================] - 3s 3ms/step - loss: 0.0043 - accuracy: 0.9991
Accuracy of the model on Training Data is -  99.91024136543274 %
464/464 [==============================] - 1s 3ms/step - loss: 0.2513 - accuracy: 0.9484
Accuracy of the model on Testing Data is -  94.8370099067688 %


In [ ]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.6, 1,0)

In [ ]:
# printing classification report

from sklearn.metrics import classification_report, f1_score


print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      7760
           1       0.95      0.94      0.94      7057

    accuracy                           0.95     14817
   macro avg       0.95      0.95      0.95     14817
weighted avg       0.95      0.95      0.95     14817



# (2) Training model with SVM

In [ ]:
from sklearn.svm import LinearSVC

# Create the LinearSVC model with some regularization
LSVC = LinearSVC(random_state=1, dual=False)

# Fit the model
LSVC.fit(X_train, y_train)

LinearSVC(dual=False, random_state=1)

In [ ]:
# Use the trained model to predict
y_pred = LSVC.predict(X_test)

In [ ]:
# model accuracy


print(f'Model train accuracy: {LSVC.score(X_train, y_train)*100:.3f}%')
print(f'Model test accuracy: {LSVC.score(X_test, y_test)*100:.3f}%')

Model train accuracy: 60.291%
Model test accuracy: 59.506%


In [ ]:
y_pred_svm = LSVC.predict(X_test)

In [ ]:
# printing classification report
print(classification_report(y_test,y_pred_svm))

              precision    recall  f1-score   support

           0       0.60      0.65      0.63      7760
           1       0.58      0.53      0.55      7057

    accuracy                           0.60     14817
   macro avg       0.59      0.59      0.59     14817
weighted avg       0.59      0.60      0.59     14817



# (3) Training model with Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the RandomForestClassifier model
rfc = RandomForestClassifier(random_state=1)

# Fit the model
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [ ]:
# Use the trained model to predict
y_pred_rfc = rfc.predict(X_test)

In [ ]:
# model accuracy
print(f'Model train accuracy: {rfc.score(X_train, y_train)*100:.3f}%')
print(f'Model test accuracy: {rfc.score(X_test, y_test)*100:.3f}%')

Model train accuracy: 99.993%
Model test accuracy: 88.081%


In [ ]:
# printing classification report
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      7760
           1       0.84      0.93      0.88      7057

    accuracy                           0.88     14817
   macro avg       0.88      0.88      0.88     14817
weighted avg       0.89      0.88      0.88     14817



**The analysis provides evidence that:**


*   Considering the f1 scores, LSTM, SVM and Random Forest Classifier methods, LSTM produced the best results.

*   In case of accuracy also, LSTM produced the best result on testing data (94.83%)


